## Dataset Load & Import

In [ ]:
!pip install sentence-transformers
!pip install faiss-gpu
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
import os
import glob
import pandas as pd
import json
import random
from sentence_transformers import SentenceTransformer, util, losses, InputExample
from torch.utils.data import DataLoader, Dataset
import faiss
import numpy as np
import wandb
import re
from collections import Counter
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 'labeldata' 폴더 및 하위 폴더 내 모든 JSON 파일 경로 가져오기 (재귀적 탐색)
json_files = glob.glob('./drive/MyDrive/멋사/labeldata/TL/**/*.json', recursive=True)

# 각 JSON 파일을 읽어 DataFrame으로 변환하고 이를 병합
all_data = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)  # JSON 파일 로드
        # JSON 데이터를 DataFrame으로 변환 (필요한 경우, 'data' 구조에 맞게 조정)
        df = pd.json_normalize(data['data'])  # 중첩된 JSON을 평탄화하여 DataFrame으로 변환
        all_data.append(df)

# 모든 DataFrame을 하나로 병합
train_df = pd.concat(all_data, ignore_index=True)


In [ ]:
# 위와 같음 (test data set)
json_files = glob.glob('./drive/MyDrive/멋사/vallabeldata/**/*.json', recursive=True)

all_data = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        df = pd.json_normalize(data['data'])
        all_data.append(df)

test_df = pd.concat(all_data, ignore_index=True)

In [ ]:
train_df.head(5)

,db_id,utterance_id,hardness,utterance_type,query,utterance,values,cols
0,seouldata_transportation_5192,Whr_11226,easy,BR03,SELECT LONGITUDE FROM TB_OPENDATA_FIXEDCCTV_J_...,고정형 CCTV 지번 주소가 을지로7가 57인 곳의 경도가 뭐야,"[{'token': '을지로7가 57', 'start': 16, 'column_in...","[{'token': '경도', 'start': 29, 'column_index': 3}]"
1,seouldata_transportation_5192,Whr_11227,easy,BR03,SELECT LATITUDE FROM TB_OPENDATA_FIXEDCCTV_J_G...,초동 공영주차장의 위도를 알려줘,"[{'token': '초동 공영주차장', 'start': 0, 'column_ind...","[{'token': '위도', 'start': 10, 'column_index': 2}]"
2,seouldata_transportation_5192,Whr_11228,easy,BR03,SELECT ADRES FROM TB_OPENDATA_FIXEDCCTV_J_G WH...,단속 지점명이 남산초교인 고정형 CCTV의 지번 주소가 어디야,"[{'token': '남산초교', 'start': 8, 'column_index':...","[{'token': '고정형 CCTV의 지번 주소', 'start': 14, 'co..."
3,seouldata_transportation_5192,Whr_11229,easy,BR03,SELECT ADRES FROM TB_OPENDATA_FIXEDCCTV_J_G WH...,불법주정차 위반 단속 감시카메라가 설치된 중구 열린애드 지점의 주소를 알려줘,"[{'token': '열린애드', 'start': 26, 'column_index'...","[{'token': '주소', 'start': 35, 'column_index': 1}]"
4,seouldata_transportation_5192,Whr_11230,medium,BR03,SELECT ADRES FROM TB_OPENDATA_FIXEDCCTV_J_G WH...,충무로에 있는 CCTV의 지번주소를 알려줘,"[{'token': '충무로', 'start': 0, 'column_index': 1}]","[{'token': 'CCTV의 지번주소', 'start': 8, 'column_i..."


In [ ]:
test_df.head(5)

,db_id,utterance_id,hardness,utterance_type,query,utterance,values,cols
0,publicdata_healthcare_740,Wht_10645,medium,BR04,SELECT CONTACT FROM GYEONGNAM_HEALTH_CLINIC WH...,소재지에 산양읍이 들어가는 보건 진료소의 연락처를 알려줘,[],"[{'token': '연락처', 'start': 23, 'column_index':..."
1,publicdata_healthcare_740,Hch_10647,medium,BR08,"SELECT CITIES, COUNT(HEALTH_CARE_CALL) FROM GY...",시군별 진료소의 개수와 시군을 보여줘,[],"[{'token': '시군', 'start': 13, 'column_index': ..."
2,publicdata_healthcare_740,Wht_10646,easy,BR04,SELECT FAX FROM GYEONGNAM_HEALTH_CLINIC WHERE ...,사천시에 있는 보건 진료소의 팩스 번호를 알려줘,"[{'token': '사천시', 'start': 0, 'column_index': 1}]","[{'token': '팩스 번호', 'start': 16, 'column_index..."
3,publicdata_healthcare_740,Wht_10647,easy,BR04,SELECT CONTACT FROM GYEONGNAM_HEALTH_CLINIC WH...,학림 보건진료소의 연락처를 알려줘,"[{'token': '학림', 'start': 0, 'column_index': 2}]","[{'token': '연락처', 'start': 10, 'column_index':..."
4,publicdata_healthcare_740,Whr_12929,easy,BR03,SELECT LOCATION FROM GYEONGNAM_HEALTH_CLINIC W...,진주시가 아닌 보건 진료소의 소재지를 알려줘,"[{'token': '진주시', 'start': 0, 'column_index': 1}]","[{'token': '소재지', 'start': 16, 'column_index':..."


In [ ]:
print(train_df.shape, test_df.shape)

(88946, 8) (11026, 8)


## RAG를 위한 벡터스토어 생성

In [ ]:
# JSON 데이터가 저장된 디렉토리 경로
json_dir_path = './drive/MyDrive/멋사/jsondata'
test_dir_path = './drive/MyDrive/멋사/valdata'
schema_info = []

# 모든 JSON 파일에 대해 반복
for filename in os.listdir(json_dir_path):
    # JSON 파일만 처리
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir_path, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            datadict = json.load(f)

        # 우리가 가지고 있는 json 구조에 따라서 진행
        datas = datadict.get('data', [])
        i = 0
        for data in datas:
          table_name = data.get('table_names_original', '')
          ko_table_name = data.get('table_names','')
          raw_columns = data.get('column_names_original', [])
          ko_raw_columns = data.get('column_names',[])
          column_names = list(map(lambda x : x[1], raw_columns))
          ko_column_names = list(map(lambda x : x[1], ko_raw_columns))
          column_types = data.get('column_types', [])
          db_id = data.get('db_id')

          schema_info.append({
            "db_id": db_id,
            "table": [table_name[0], ko_table_name[0]],
            "column_names": [column_names,ko_column_names],
            "column_types": column_types
          })

# 위와 같음 (test data set)
for filename in os.listdir(test_dir_path):
    if filename.endswith('.json'):
        file_path = os.path.join(test_dir_path, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            datadict = json.load(f)

        datas = datadict.get('data', [])
        i = 0
        for data in datas:
          table_name = data.get('table_names_original', '')
          ko_table_name = data.get('table_names','')
          raw_columns = data.get('column_names_original', [])
          ko_raw_columns = data.get('column_names',[])
          column_names = list(map(lambda x : x[1], raw_columns))
          ko_column_names = list(map(lambda x : x[1], ko_raw_columns))
          column_types = data.get('column_types', [])
          db_id = data.get('db_id')

          schema_info.append({
            "db_id": db_id,
            "table": [table_name[0], ko_table_name[0]],
            "column_names": [column_names,ko_column_names],
            "column_types": column_types
          })

In [ ]:
print(schema_info[0])

{'db_id': 'seouldata_healthcare_455', 'table': ['SEOUL_PUBLIC_HYGIENE_BIZ', '서울시 기타 위생용품 제조업 현황'], 'column_names': [['*', 'CGG_CODE', 'SNT_COB_CODE', 'YY', 'UPSO_SNO', 'SNT_COB_NM', 'UPSO_GSL_YMD', 'UPSO_NM', 'TRDP_AREA', 'UPSO_SITE_TELNO', 'BMAN_STDT', 'BUP_NM', 'SITE_STDT', 'ADMDNG_NM', 'DCB_YMD', 'ED_FIN_YMD', 'GAEKSIL', 'HANSHIL', 'YANGSIL', 'CHAIR_NUM', 'YOKSIL', 'BALHANSIL_YN', 'PERM_NT_NO', 'SITE_ADDR_RD'], ['*', '시군구코드', '업종코드', '년도', '업소일련번호', '업종명', '신고일자', '업소명', '영업장면적 제곱미터', '소재지전화번호', '영업자시작일', '법인명', '소재지시작일', '행정동명', '폐업일자', '위생교육수료일자', '객실수', '한실수', '양실수', '의자수', '욕실수', '발한실', '허가 신고 번호', '소재지도로명']], 'column_types': ['text', 'number', 'number', 'time', 'number', 'text', 'time', 'text', 'number', 'text', 'time', 'text', 'time', 'text', 'time', 'time', 'text', 'text', 'text', 'text', 'text', 'text', 'text', 'text']}


In [ ]:
len(schema_info)

5761

In [ ]:
# schema list 생성, db_id 기반으로 검색
train_schemas = []
for id in train_df['db_id']:
  for i in range(len(schema_info)):
    if schema_info[i]['db_id'] == id:
      schema_text = f"Table: {schema_info[i]['table'][0]} Table, Columns: {', '.join([f'{col} Columns' for col in schema_info[i]['column_names'][0]])}"
      train_schemas.append(schema_text)

train_df['schema'] = train_schemas

In [ ]:
test_schemas = []
for id in test_df['db_id']:
  for i in range(len(schema_info)):
    if schema_info[i]['db_id'] == id:
      schema_text = f"Table: {schema_info[i]['table'][0]} Table, Columns: {', '.join([f'{col} Columns' for col in schema_info[i]['column_names'][0]])}"
      test_schemas.append(schema_text)

test_df['schema'] = test_schemas

In [ ]:
train_df['schema'].head(5)

,schema
0,"Table: TB_OPENDATA_FIXEDCCTV_J_G Table, Column..."
1,"Table: TB_OPENDATA_FIXEDCCTV_J_G Table, Column..."
2,"Table: TB_OPENDATA_FIXEDCCTV_J_G Table, Column..."
3,"Table: TB_OPENDATA_FIXEDCCTV_J_G Table, Column..."
4,"Table: TB_OPENDATA_FIXEDCCTV_J_G Table, Column..."


In [ ]:
train_df['schema'].nunique()

5118

## Embedding model Train

In [ ]:
# 양성 샘플 schema list 생성, db_id 기반으로 검색
pos_schemas = []
for id in train_df['db_id']:
  for i in range(len(schema_info)):
    if schema_info[i]['db_id'] == id:
      schema_text = f"Table: {schema_info[i]['table'][0]} / {schema_info[i]['table'][1]}, Columns: {', '.join([f'{col}/{col2} (Type: {type})' for col, col2, type in zip(schema_info[i]['column_names'][0], schema_info[i]['column_names'][1], schema_info[i]['column_types'])])}"
      pos_schemas.append(schema_text)

In [ ]:
# 음성 샘플 schema list 생성, id 같지 않은 것들 중 랜덤 샘플링
neg_schemas = []
for id in train_df['db_id']:
  random_schema = random.choice(schema_info)
  while random_schema['db_id'] == id:
    random_schema = random.choice(schema_info)
  schema_text = f"Table: {random_schema['table'][0]} / {random_schema['table'][1]}, Columns: {', '.join([f'{col}/{col2} (Type: {type})' for col, col2, type in zip(random_schema['column_names'][0], random_schema['column_names'][1], random_schema['column_types'])])}"
  neg_schemas.append(schema_text)

In [ ]:
print(train_df['utterance'][0])
print(len(pos_schemas), pos_schemas[0])
print(len(neg_schemas), neg_schemas[0])

초동 공영주차장의 위도를 알려줘
88946 Table: TB_OPENDATA_FIXEDCCTV_J_G / 서울시 중구 불법주정차 위반 단속 CCTV 위치정보, Columns: */* (Type: text), ADRES/고정형CCTV지번주소 (Type: text), LATITUDE/위도 (Type: number), LONGITUDE/경도 (Type: number), PSTINST_CD_NM/중구 자치구 (Type: text), REGLT_SPOT_NM/단속지점명 (Type: text)
88946 Table: L_O_C_A_L_D_A_T_A_093008_D_B / 서울시 도봉구 대기오염물질배출시설설치사업장 인허가 정보, Columns: */* (Type: text), OPNSFTEAMCODE/개방자치단체코드 (Type: number), MGTNO/관리번호 (Type: number), APVPERMYMD/인허가일자 (Type: time), APVCANCELYMD/인허가취소일자 (Type: text), TRDSTATEGBN/영업상태코드 (Type: number), TRDSTATENM/영업상태명 (Type: text), CLGSTDT/휴업시작일자 (Type: text), ROPNYMD/재개업일자 (Type: text), SITETEL/전화번호 (Type: text), SITEWHLADDR/지번주소 (Type: text), RDNPOSTNO/도로명우편번호 (Type: number), BPLCNM/사업장명 (Type: text), LASTMODTS/최종수정일자 (Type: number), UPDATEGBN/데이터갱신구분 (Type: text), UPDATEDT/데이터갱신일자 (Type: time), X/좌표정보 X (Type: text), Y/좌표정보 Y (Type: text), ENVBSNSENM/환경업무구분명 (Type: text), COBGBNNM/업종구분명 (Type: text), JONGGBNNM/종별명 (Type: text), PDTATCLNM/주생산품명 (Ty

In [ ]:
queries = train_df['utterance'].tolist()  # 쿼리 리스트
positive_samples = []
negative_samples = []

# 리스트 길이 확인
if (len(queries) != len(pos_schemas)) or (len(queries) != len(neg_schemas)):
    raise ValueError("queries와 schemas의 길이가 다릅니다. 데이터셋을 확인하세요.")

for i in range(len(pos_schemas)):
    positive_samples.append((queries[i], pos_schemas[i], 1))  # 라벨 1은 양성 샘플

for i in range(len(neg_schemas)):
    negative_samples.append((queries[i], neg_schemas[i], 0))  # 라벨 0은 음성 샘플

# 결과 출력
print(f"양성 샘플 개수: {len(positive_samples)}")
print(f"음성 샘플 개수: {len(negative_samples)}")


양성 샘플 개수: 88946
음성 샘플 개수: 88946


In [ ]:
# 모델 로드
model = SentenceTransformer('intfloat/multilingual-e5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
# W&B 설정 제거
wandb.init(mode="disabled")

In [ ]:
# 양성 샘플 & 음성 샘플 결합
train_samples = positive_samples + negative_samples
# train_data 형식에 맞게 변형
train_data = [InputExample(texts=[query, schema], label=float(label)) for query, schema, label in train_samples]

# 모델 학습을 위한 DataLoader
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

# Contrastive Loss 사용
train_loss = losses.MultipleNegativesRankingLoss(model=model)

# 모델 학습
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100)

# 모델 저장
model.save('query_schema_model')


Step,Training Loss
500,1.521300
1000,1.461300
1500,1.400700
2000,1.423800
2500,1.419300
3000,1.409400
3500,1.400100
4000,1.361000
4500,1.397400
5000,1.333800


Step,Training Loss
500,1.521300
1000,1.461300
1500,1.400700
2000,1.423800
2500,1.419300
3000,1.409400
3500,1.400100
4000,1.361000
4500,1.397400
5000,1.333800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
model_save_path = '/content/drive/MyDrive/query_schema_model'

# 모델 저장
model.save(model_save_path)

print(f"Model saved at {model_save_path}")

Model saved at /content/drive/MyDrive/query_schema_model


## 추론

In [ ]:
# 임베딩 모델 로드(학습한 모델)
model = SentenceTransformer('/content/drive/MyDrive/query_schema_model')

# 스키마 정보를 텍스트로 변환하여 임베딩 생성
schema_texts = [
    f"Table: {table['table'][0]} / {table['table'][1]}, Columns: {', '.join([f'{col}/{col2} (Type: {type})' for col, col2, type in zip(table['column_names'][0], table['column_names'][1], table['column_types'])])}"
    for table in schema_info
]
schema_embeddings = model.encode(schema_texts)

# FAISS 인덱스 생성 및 벡터 추가
dimension = schema_embeddings.shape[1]  # 임베딩 차원 수
index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스 생성
index.add(np.array(schema_embeddings))  # 임베딩 추가

print("FAISS 인덱스에 스키마 정보 벡터화 및 저장 완료")


FAISS 인덱스에 스키마 정보 벡터화 및 저장 완료


In [ ]:
# 벡터 크기 확인 (모델의 벡터 크기: 768차원)
print("벡터 크기:", schema_embeddings.shape)  # 결과: (1, 768)
# 첫 번째 문장 스키마 텍스트 확인
print("첫 번째 문장 스키마 텍스트:", schema_texts[0])
# 첫 번째 문장의 벡터 일부 확인
print("첫 번째 문장의 벡터 값:", schema_embeddings[0][:10])  # 첫 10개 요소 출력

벡터 크기: (5121, 768)
첫 번째 문장 스키마 텍스트: Table: SEOUL_PUBLIC_HYGIENE_BIZ / 서울시 기타 위생용품 제조업 현황, Columns: */* (Type: text), CGG_CODE/시군구코드 (Type: number), SNT_COB_CODE/업종코드 (Type: number), YY/년도 (Type: time), UPSO_SNO/업소일련번호 (Type: number), SNT_COB_NM/업종명 (Type: text), UPSO_GSL_YMD/신고일자 (Type: time), UPSO_NM/업소명 (Type: text), TRDP_AREA/영업장면적 제곱미터 (Type: number), UPSO_SITE_TELNO/소재지전화번호 (Type: text), BMAN_STDT/영업자시작일 (Type: time), BUP_NM/법인명 (Type: text), SITE_STDT/소재지시작일 (Type: time), ADMDNG_NM/행정동명 (Type: text), DCB_YMD/폐업일자 (Type: time), ED_FIN_YMD/위생교육수료일자 (Type: time), GAEKSIL/객실수 (Type: text), HANSHIL/한실수 (Type: text), YANGSIL/양실수 (Type: text), CHAIR_NUM/의자수 (Type: text), YOKSIL/욕실수 (Type: text), BALHANSIL_YN/발한실 (Type: text), PERM_NT_NO/허가 신고 번호 (Type: text), SITE_ADDR_RD/소재지도로명 (Type: text)
첫 번째 문장의 벡터 값: [-0.03422691 -0.00850079 -0.05123332 -0.04028866  0.00320186  0.00627503
  0.01396287 -0.03120078 -0.07215644 -0.01271592]


In [ ]:
# 사용자 질문 예시(train_data)
for i in range(100):
  sample = train_df[['query','utterance']].sample(n=1, random_state=i).values[0]
  query = sample[1]
  print(f"입력 쿼리 문 : {query}")
  query_embedding = model.encode([query])

  # FAISS 인덱스에서 유사한 스키마 검색
  k = 1  # 상위 1개의 관련 결과를 검색
  distances, indices = index.search(np.array(query_embedding), k)

  print("가장 관련성이 높은 스키마 정보:")
  for idx in indices[0]:
      print(schema_texts[idx])
  print(f'정답 : {sample[0]}')
  print('-'*100)

입력 쿼리 문 : 불광동에 있는 유료직업소개소의 X 좌푯값과 Y 좌푯값을 알려줘
가장 관련성이 높은 스키마 정보:
Table: L_O_C_A_L_D_A_T_A_115002_Y_D / 서울시 영등포구 유료직업소개소 인허가 정보, Columns: */* (Type: text), OPNSFTEAMCODE/개방자치단체코드 (Type: number), MGTNO/관리번호 (Type: number), APVPERMYMD/인허가일자 (Type: time), APVCANCELYMD/인허가취소일자 (Type: text), TRDSTATEGBN/영업상태코드 (Type: number), TRDSTATENM/영업상태명 (Type: text), DTLSTATEGBN/상세영업상태코드 (Type: number), DTLSTATENM/상세영업상태명 (Type: text), DCBYMD/폐업일자 (Type: time), CLGSTDT/휴업시작일자 (Type: text), CLGENDDT/휴업종료일자 (Type: text), ROPNYMD/재개업일자 (Type: text), SITEAREA/소재지면적 (Type: text), SITEPOSTNO/소재지우편번호 (Type: text), SITEWHLADDR/지번주소 (Type: text), RDNWHLADDR/도로명주소 (Type: text), RDNPOSTNO/도로명우편번호 (Type: number), BPLCNM/사업장명 (Type: text), LASTMODTS/최종수정일자 (Type: number), UPDATEGBN/데이터갱신구분 (Type: text), UPDATEDT/데이터갱신일자 (Type: time), UPTAENM/업태구분명 (Type: text), X/좌표정보 X (Type: text), Y/좌표정보 Y (Type: text), BUPGBNNM/법인구분명 (Type: text), SENM/구분명 (Type: text)
정답 : SELECT X, Y FROM L_O_C_A_L_D_A_T_A_115002_E_P WHERE SIT

In [ ]:
# 사용자 질문 예시(val_data)
for i in range(100):
  sample = test_df[['query','utterance']].sample(n=1, random_state=i).values[0]
  query = sample[1]
  print(f"입력 쿼리 문 : {query}")
  query_embedding = model.encode([query])

  # FAISS 인덱스에서 유사한 스키마 검색
  k = 2  # 상위 2개의 관련 결과를 검색
  distances, indices = index.search(np.array(query_embedding), k)

  print("가장 관련성이 높은 스키마 정보:")
  for idx in indices[0]:
      print(schema_texts[idx])
  print(f'정답 : {sample[0]}')
  print('-'*100)

입력 쿼리 문 : 차시명에 무선 통신망이라는 말이 들어간 콘텐츠의 제작일을 중복 없이 보여줘
가장 관련성이 높은 스키마 정보:
Table: ANNUAL_S_MENIER_MANUFACTURER_GREENHOUSE_GAS_EMISSION_S_P / 환경부 국립환경과학원 제작사별연간판매자동차온실가스배출기준및실적, Columns: */* (Type: text), NATIOND/국가 (Type: text), MANUFACTURING_COMPANYE/제작 업체 (Type: text), SALES_YEARF/판매 연도 (Type: number), SELECTION_CRITERIAG/선택 기준 (Type: text), SALES_SIZEH/판매 규모 (Type: text), A_S_V_T_C/적용기준 차종 (Type: text), SALES_VOLUME_CHECKJ/판매량 확인 (Type: text), PROGRESS_STAK/진행 상태 (Type: text), P_C_S_F/실적 확인 상태 (Type: text), G_G_R_R_G/기준 비율별 온실가스 (Type: text), E_C_E_R_R_H/기준 비율별 에너지소비효율 (Type: text), G_G_B_A_B_I/기준 연차별 온실가스 (Type: number), E_C_E_P_S_Y_J/기준 연차별 에너지소비효율 (Type: number), P_1_G_G_K/실적 100 온실가스 (Type: text), P_1_E_C_E_L/실적 100 에너지소비효율 (Type: text), P_R_G_G_M/실적 비율별 온실가스 (Type: text), P_R_E_C_E_N/실적 비율별 에너지소비효율 (Type: text), SALES_VS_SALESU/비율 판매량 (Type: number), SALES_VS_OVERALL_SALESV/전체 판매량 (Type: number), AVG_TOLERANCE_WEIGHTW/평균 공차 중량 (Type: text), AVG_OCCUPIED_AREAX/평균 점유 면적 (Type: text),

## 단어사전 구성

In [ ]:
utterances = train_df['utterance'].tolist() + test_df['utterance'].tolist()
schemas =  train_df['schema'].tolist() + test_df['schema'].tolist()
queries = train_df['query'].tolist() + test_df['query'].tolist()

In [ ]:
utterances[-1]

'수서동에 있는 남자의 나이대를 인구수가 적은 순으로 정렬해 줘'

In [ ]:
schemas[-1]

'Table: POP_003 Table, Columns: * Columns, STATS_YR Columns, SIGUNGU_NM Columns, EPMNDN_NM Columns, SEX_NM Columns, AGEGRD_NM Columns, POPL_CNT Columns, ADMDONG_CD Columns, AGEGRD_CD Columns'

In [ ]:
queries[-1]

"SELECT AGEGRD_NM FROM POP_003 WHERE EPMNDN_NM = '수서동' AND SEX_NM = '남자' ORDER BY POPL_CNT"

In [ ]:
#입력된 text를 소문자로 변환한 후,
#정규표현식을 사용하여 단어와 특수 문자를 분리하여 리스트 형태로 반환

def tokenize_and_replace(text):
    # 소문자로 변환하고 'table'과 'columns'을 각각 'T'와 'C'로 대체
    text = text.lower().replace("table", "T").replace("columns", "C").replace('type', 'P')
    # 정규표현식을 사용해 단어와 특수기호를 분리, 이후 특정 특수기호는 제거
    tokens = re.findall(r"\w+|\S", text)
    # 제거할 특수기호를 제외하고 토큰 필터링
    tokens = [token for token in tokens if token not in {',', ':', '/', '(', ')'}]
    return tokens

In [ ]:
print(tokenize_and_replace(utterances[-1]))

['수서동에', '있는', '남자의', '나이대를', '인구수가', '적은', '순으로', '정렬해', '줘']


In [ ]:
print(tokenize_and_replace(schemas[-1]))

['T', 'pop_003', 'T', 'C', '*', 'C', 'stats_yr', 'C', 'sigungu_nm', 'C', 'epmndn_nm', 'C', 'sex_nm', 'C', 'agegrd_nm', 'C', 'popl_cnt', 'C', 'admdong_cd', 'C', 'agegrd_cd', 'C']


In [ ]:
print(tokenize_and_replace(queries[-1]))

['select', 'agegrd_nm', 'from', 'pop_003', 'where', 'epmndn_nm', '=', "'", '수서동', "'", 'and', 'sex_nm', '=', "'", '남자', "'", 'order', 'by', 'popl_cnt']


In [ ]:


#텍스트 리스트를 입력받아 어휘 사전을 생성
def build_vocab(texts, min_freq):
    token_counter = Counter()
    for text in texts:
        tokens = tokenize_and_replace(text)
        token_counter.update(tokens) #토큰 빈도 계산

#token_counter에 있는 단어들을 순서대로 인덱스 값과 매핑하여
#사전 vocab에 저장합니다. 여기서 인덱스는 4부터 시작
#특수 토큰 추가: 모델 훈련에 필요한 네 가지 특수 토큰을 사전에 추가
    filtered_tokens = [(word, count) for word, count in token_counter.items() if count >= min_freq]

    vocab = {word: i+6 for i, (word, count) in enumerate(filtered_tokens)}
    vocab["<PAD>"] = 0 #빈 칸을 채우기 위한 패딩
    vocab["<SOS>"] = 1 #문장의 시작
    vocab["<EOS>"] = 2 #문장의 끝
    vocab["<UNK>"] = 3 #미등록 단어
    vocab['T'] = 4
    vocab['C'] = 5
    return vocab

src_vocab = build_vocab(utterances+schemas, 20) #utterances + schemas(입력 문장)에 대해 생성된 어휘 사전
tgt_vocab = build_vocab(queries, 20) #train_queries(SQL 쿼리)로부터 생성된 어휘 사전

In [ ]:
print(len(src_vocab),src_vocab)

17463 {'고정형': 6, 'cctv': 7, '지번': 8, '주소가': 9, '곳의': 10, '경도가': 11, '뭐야': 12, '위도를': 13, '알려줘': 14, '단속': 15, '지점명이': 16, 'cctv의': 17, '어디야': 18, '위반': 19, '설치된': 20, '중구': 21, '지점의': 22, '주소를': 23, '있는': 24, '지번주소를': 25, '불법': 26, '주정차': 27, '위치정보에서': 28, '이름에': 29, '은행이': 30, '들어가는': 31, '찾아줘': 32, '또는': 33, '보여줘': 34, '지점명을': 35, '위도와': 36, '경도를': 37, '주소에': 38, '신당동이': 39, '가나다': 40, '역순으로': 41, '정렬해줘': 42, '지점명에': 43, '들어간': 44, '가나다순으로': 45, '곳에': 46, '나타내줘': 47, '을지로가': 48, '시작하는': 49, '서울': 50, '은평구': 51, '초등학교가': 52, '포함된': 53, '주유소가': 54, '학교가': 55, '위치한': 56, '위도가': 57, '높은': 58, '순부터': 59, '나타내': 60, '줘': 61, 'cctv가': 62, '경도': 63, '위도': 64, '정보와': 65, '정보를': 66, '경도는': 67, '위치의': 68, '주소': 69, '중': 70, '아파트가': 71, '낮은': 72, '지점이': 73, '수는': 74, '얼마야': 75, '포함되는': 76, '화곡동이': 77, '이름을': 78, '및': 79, '경도와': 80, '이름이': 81, '끝나는': 82, '정보에서': 83, '주소는': 84, '중복': 85, '없이': 86, '나열해': 87, '포함되거나': 88, '소재지': 89, '도로명주소와': 90, '신청': 91, '시': 92, '방법에': 93, '정기': 94, '접수': 95, '시

In [ ]:
print(len(tgt_vocab))
print(tgt_vocab)

5296
{'select': 6, 'longitude': 7, 'from': 8, 'where': 9, 'adres': 10, '=': 11, "'": 12, '57': 13, 'latitude': 14, 'reglt_spot_nm': 15, 'like': 16, '%': 17, '충무로': 18, '은행': 19, 'or': 20, '을지로': 21, '신당동': 22, 'order': 23, 'by': 24, 'desc': 25, '부근': 26, '서울': 27, '은평구': 28, '13': 29, '앞': 30, '초등학교': 31, '주유소': 32, '학교': 33, '5': 34, '불광동': 35, '응암동': 36, '한남동': 37, '아파트': 38, '후암동': 39, '95': 40, 'count': 41, '삼거리': 42, 'positn_nm': 43, '화곡동': 44, 'distinct': 45, '동대문구': 46, '60000': 47, 'rdnmadr': 48, '장애인': 49, 'xce': 50, 'traffic_safety_a077_p_info': 51, '02': 52, '-': 53, 'mk_cpy': 54, 'esb_ymd': 55, 'cae_ymd': 56, 'yce': 57, '대한': 58, 'hisid': 59, '9': 60, '2020': 61, '2021': 62, '2000': 63, '2': 64, 'mgrnu': 65, 'group': 66, '시스템': 67, 'statn_nm': 68, '<': 69, '00': 70, 'line': 71, '1호선': 72, 'and': 73, '2호선': 74, '1': 75, '30': 76, 'avg': 77, '신당': 78, '종로': 79, 'tb_opendata_fixedcctv_s_c': 80, '주변': 81, '7': 82, '서초구': 83, '서초': 84, 'asc': 85, '66': 86, 'sgg_nm': 87, 'emd_nm'

In [ ]:
def text_to_indices(text, vocab, max_len=50):
    tokens = tokenize_and_replace(text)
    #각 토큰을 어휘 사전 vocab을 통해 인덱스로 변환
    #vocab에 해당 토큰이 존재하지 않는 경우 <UNK>(미등록 단어) 토큰의 인덱스로 대체
    #tokens[:max_len-2]로 max_len - 2 길이까지 자릅니다. 이 제한은 <SOS>와 <EOS>를 추가할 공간을 확보하기 위함
    indices = [vocab.get(token, vocab["<UNK>"]) for token in tokens[:max_len-2]]  # <UNK>로 대체
    #문장의 시작과 끝을 나타내기 위해 <SOS>와 <EOS> 토큰의 인덱스를 리스트의 앞과 뒤에 추가
    indices = [vocab["<SOS>"]] + indices + [vocab["<EOS>"]]
    #indices의 길이가 max_len보다 작을 경우, 부족한 길이만큼 <PAD> 토큰 인덱스를 추가
    if len(indices) < max_len:
        indices += [vocab["<PAD>"]] * (max_len - len(indices))
    assert all(i < len(vocab) for i in indices), "Index out of vocabulary bounds!"
    return indices

# text: 인덱스로 변환할 텍스트
# vocab: 단어를 인덱스로 매핑한 어휘 사전
# max_len: 변환된 인덱스 리스트의 최대 길이 (기본값은 50)

In [ ]:
utterances = [utterance + ' ' + ' '.join(tokenize_and_replace(schema)) for utterance, schema in zip(train_df['utterance'].tolist(), train_df['schema'].tolist())]
queries = train_df['query'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split

# 예시: utterances와 queries를 훈련 세트와 테스트 세트로 나누기
train_utterances, val_utterances, train_queries, val_queries = train_test_split(
    utterances, queries, test_size=0.2, random_state=42
)

In [ ]:
test_utterances = [utterance + ' ' + ' '.join(tokenize_and_replace(schema)) for utterance, schema in zip(test_df['utterance'].tolist(), test_df['schema'].tolist())]
test_queries = test_df['query'].tolist()

In [ ]:
train_utterances[0]

'역명이 송파인 승차인원의 1월 선불을 보여줘 T num_passengers_selmetro_winding_P T C * C annual C line C reverse_number C station_name C january_f C after_january C january_s C january_great_power C january_1_ticket C january_o C february_f C after_february C february_s C february_big_power C february_1_ticket C february_o C march_f C late_march C march_s C march_great_power C march_1_ticket C march_o C april_buddha C late_april C april_s C april_great_power C april_1_ticket C april_o C fund_in_may C later_in_may C may_s C may_great_power C may_1_ticket C may_o C'

In [ ]:
tokenize_and_replace('hello world')

['hello', 'world']

In [ ]:
print(text_to_indices(train_utterances[0], src_vocab, max_len=50))

#예를 들면
#토큰화 결과: ['hello', 'world']
#인덱스 변환: [5, 3] ("world"가 사전에 없으므로 <UNK>로 대체됨)
#<SOS>와 <EOS> 추가: [1, 5, 3, 2]
#패딩 추가: [1, 5, 3, 2, 0] (길이 5에 맞춰 <PAD> 추가)

[1, 168, 3, 3, 513, 3, 34, 3, 3, 3, 3, 4085, 3, 7045, 3, 4120, 3, 13761, 3, 13762, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2]


In [ ]:
val_utterances[0]

'배출 업체 번호가 2017로 시작하는 데이터의 운반 업체 번호를 오름차순으로 중복 없이 보여줘 T keco_ls_ex_e_handover_mng_sys_emission_mng_register_amt T C * C management_register_numw C base_dt_dischargex C emission_company_numy C carrier_numz C fertilizer_clasa C processing_clasb C processing_method_cdc C emissionsd C register_re_Pe C'

In [ ]:
val_queries[0]

"SELECT DISTINCT CARRIER_NUMZ FROM KECO_LS_EX_E_HANDOVER_MNG_SYS_EMISSION_MNG_REGISTER_AMT WHERE EMISSION_COMPANY_NUMY LIKE '2017%' ORDER BY CARRIER_NUMZ"

## Dataset Load

In [ ]:
class TextToSQLDataset(Dataset):
    def __init__(self, utterances, queries, src_vocab, tgt_vocab, max_len=100):
        self.utterances = utterances #학습할 텍스트 입력 리스트 (예: 자연어 질문)
        self.queries = queries #해당 텍스트에 대한 SQL 쿼리 리스트
        self.src_vocab = src_vocab #입력 텍스트(utterance)의 어휘 사전
        self.tgt_vocab = tgt_vocab #SQL 쿼리(query)의 어휘 사전
        self.max_len = max_len #인덱스 리스트의 최대 길이 (기본값은 50)

    #데이터셋의 샘플 개수를 반환합니다.
    #이는 데이터셋의 길이를 정의하여 DataLoader가 데이터의 총 개수를 알 수 있도록 합니다.
    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, idx):
        #입력 텍스트 인덱싱: elf.utterances[idx] 텍스트를 src_vocab 사전을 사용하여 인덱스 리스트(src_indices)로 변환
        src_indices = text_to_indices(self.utterances[idx], self.src_vocab, self.max_len)
        #SQL 쿼리 인덱싱 : self.queries[idx] 텍스트를 tgt_vocab 사전을 사용해 인덱스 리스트(tgt_indices)로 변환
        tgt_indices = text_to_indices(self.queries[idx], self.tgt_vocab, self.max_len)
        #텐서 변환:src_indices와 tgt_indices 리스트를 torch.tensor로 변환하여 반환

        # 타겟의 경우, 텍스트의 길이를 맞추기 위해 패딩을 추가할 수 있으므로 -100으로 패딩 처리
        labels = torch.tensor(tgt_indices, dtype=torch.long)
        labels[labels == self.tgt_vocab.get('<pad>', 0)] = -100  # 패딩 부분은 -100으로 처리

        return torch.tensor(src_indices), labels


In [ ]:
len(train_utterances)

71156

In [ ]:
# 데이터셋 생성
train_dataset = TextToSQLDataset(train_utterances, train_queries, src_vocab, tgt_vocab)
val_dataset = TextToSQLDataset(val_utterances, val_queries, src_vocab, tgt_vocab)

In [ ]:
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
optimizer = AdamW(model.parameters(),lr= 5e-5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
for batch in train_loader:
    input_ids = batch[0].to(device)
    labels = batch[1].to(device)

    print(f"Input IDs: {input_ids}")
    print(f"Labels: {labels}")
    break  # 첫 번째 배치만 확인해봄

Input IDs: tensor([[   1,    3,  738,  ...,    0,    0,    0],
        [   1, 1824, 1826,  ...,    0,    0,    0],
        [   1, 1367,    3,  ...,    0,    0,    0],
        ...,
        [   1,    3,    3,  ...,    0,    0,    0],
        [   1, 1801,  755,  ...,    0,    0,    0],
        [   1, 4030,    3,  ...,    0,    0,    0]], device='cuda:0')
Labels: tensor([[   1,    6, 2728,  ..., -100, -100, -100],
        [   1,    6,  274,  ..., -100, -100, -100],
        [   1,    6, 2149,  ..., -100, -100, -100],
        ...,
        [   1,    6,   45,  ..., -100, -100, -100],
        [   1,    6, 3372,  ..., -100, -100, -100],
        [   1,    6,    3,  ..., -100, -100, -100]], device='cuda:0')


In [ ]:
from tqdm import tqdm

def save_checkpoint(model, optimizer, epoch, loss, filename="checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved at epoch {epoch}!")

# 학습 루프
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader,desc=f"Epoch {epoch+1}/{epochs}"):
        # 배치 데이터 가져오기
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        assert (labels < len(tgt_vocab)).all(), "Label indices are out of vocabulary bounds."


        # 모델 학습
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)

        # 손실 계산
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 에폭마다 손실 출력
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

    if (epoch + 1) % 2 == 0:
        save_checkpoint(model, optimizer, epoch, total_loss/len(train_loader), filename=f"./drive/MyDrive/멋사/checkpoint_epoch_{epoch+1}.pth")


Epoch 1/5: 100%|██████████| 2224/2224 [03:57<00:00,  9.36it/s]


Epoch 1/5, Loss: 2.0074523972521585


Epoch 2/5: 100%|██████████| 2224/2224 [03:57<00:00,  9.37it/s]


Epoch 2/5, Loss: 1.8142920241510267
Checkpoint saved at epoch 1!


Epoch 3/5: 100%|██████████| 2224/2224 [03:57<00:00,  9.36it/s]


Epoch 3/5, Loss: 1.6831779687417496


Epoch 4/5: 100%|██████████| 2224/2224 [03:57<00:00,  9.36it/s]


Epoch 4/5, Loss: 1.5928805569093005
Checkpoint saved at epoch 3!


Epoch 5/5: 100%|██████████| 2224/2224 [03:57<00:00,  9.36it/s]

Epoch 5/5, Loss: 1.5213622544630825


In [ ]:
def save_model_to_path(model, save_path):
    # 모델의 state_dict를 저장합니다.
    torch.save(model.state_dict(), save_path)
    print(f"Model saved at {save_path}")


In [ ]:
checkpoint = torch.load("checkpoint_epoch_4.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # 추론 모드로 전환

# 원하는 경로에 다시 저장
save_model_to_path(model, save_path="./drive/MyDrive/멋사/model.pth")

<ipython-input-47-6c8c823ce3fd>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("checkpoint_epoch_4.pth", map_location=torch.device('cuda' if torch.

Model saved at ./drive/MyDrive/멋사/model.pth


In [ ]:
def load_model_for_inference(model, filename="final_model.pth"):
    # 체크포인트 파일을 로드합니다.
    checkpoint = torch.load(filename, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    # 모델의 가중치를 로드합니다.
    model.load_state_dict(checkpoint)

    # 모델을 평가 모드로 전환합니다.
    model.eval()

    print("Model loaded and set to inference mode.")
    return model



In [ ]:
model = load_model_for_inference(model, filename="./drive/MyDrive/멋사/model.pth")

<ipython-input-52-624b854dda94>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map_location=torch.device('cuda' if torch.cuda.is_availabl

Model loaded and set to inference mode.


In [ ]:
def generate_sql_query(utterance, src_vocab, tgt_vocab, model):
    # 입력 텍스트를 인덱스로 변환
    input_tokens = text_to_indices(utterance, src_vocab)
    input_ids = torch.tensor(input_tokens).unsqueeze(0).to(device)  # 배치 차원 추가

    # 모델로부터 예측값 생성
    output_ids = model.generate(input_ids=input_ids, max_length=20)
    print(output_ids)

    # 예측 결과를 SQL 쿼리로 디코딩
    sql_query = " ".join([list(tgt_vocab.keys())[list(tgt_vocab.values()).index(id.item())] for id in output_ids[0]])

    return sql_query

# 예시
utterance = val_utterances[5]
generated_query = generate_sql_query(utterance, src_vocab, tgt_vocab, model)
print(f"Input Text: {utterance}")
print(f"Generated SQL Query: {generated_query}")


tensor([[0, 1]], device='cuda:0')
Input Text: 상세 영업 상태명이 폐업인 곳의 사무실 면적을 나타내줘 T l_o_c_a_l_d_a_t_a_093011_g_d T C * C opnsfteamcode C mgtno C apvpermymd C trdstategbn C trdstatenm C dtlstategbn C dtlstatenm C dcbymd C clgstdt C clgenddt C ropnymd C sitetel C sitepostno C rdnwhladdr C rdnpostno C bplcnm C updategbn C updatedt C uptaenm C x C y C ofear C disfetvehgarar C microspklnum C hndusestlznum C dynpwspraynum C hdoptdspraynum C gmknum C protuseclotnum C vacuclernum C
Generated SQL Query: <PAD> <SOS>
